In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.50.15.159:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1582553000651)
SparkSession available as 'spark'


import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


In [2]:
val inputStreamPath = "./data/events-stream"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = ./data/events-stream
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [3]:
val trainedModel = PipelineModel.load("./models/spark-ml-model-rf")
val getProbabilityNegativeTweet = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
val getProbabilityPositiveTweet = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(0))

trainedModel: org.apache.spark.ml.PipelineModel = pipeline_17a6fd2fa90f
getProbabilityNegativeTweet: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
getProbabilityPositiveTweet: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


In [4]:
val predictionDf = trainedModel.transform(inputDF)

predictionDf: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 6 more fields]


In [5]:
val markedPositiveDf = predictionDf
        .withColumn("probability_negative", getProbabilityNegativeTweet($"probability"))
        .withColumn("probability_positive", getProbabilityPositiveTweet($"probability"))
        .withColumn("is_positive", when($"probability_positive" > 0.5, 1).otherwise(0))

val timedDf = markedPositiveDf
    .withColumn("last_minute_timestamp", (unix_timestamp().minus(1 * 60) + rand() * 100).cast("timestamp"))

val countIsPositivePer10Sec = timedDf
    .withWatermark("last_minute_timestamp", "1 seconds")
    .groupBy(
        col("is_positive"),
        window(col("last_minute_timestamp"), "10 seconds", "10 seconds")
    ).count()

markedPositiveDf: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 9 more fields]
timedDf: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 10 more fields]
countIsPositivePer10Sec: org.apache.spark.sql.DataFrame = [is_positive: int, window: struct<start: timestamp, end: timestamp> ... 1 more field]


In [6]:
countIsPositivePer10Sec.writeStream.foreachBatch{ (batchDF: DataFrame, batchId: Long) =>
    batchDF.show(false)
}.start()

res0: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@17c904ef


+-----------+------+-----+
|is_positive|window|count|
+-----------+------+-----+
+-----------+------+-----+

+-----------+------------------------------------------+-----+
|is_positive|window                                    |count|
+-----------+------------------------------------------+-----+
|1          |[2020-02-24 17:02:50, 2020-02-24 17:03:00]|1    |
|0          |[2020-02-24 17:04:00, 2020-02-24 17:04:10]|1    |
|0          |[2020-02-24 17:03:50, 2020-02-24 17:04:00]|2    |
|0          |[2020-02-24 17:03:20, 2020-02-24 17:03:30]|1    |
|1          |[2020-02-24 17:02:30, 2020-02-24 17:02:40]|1    |
|1          |[2020-02-24 17:03:40, 2020-02-24 17:03:50]|1    |
|1          |[2020-02-24 17:02:40, 2020-02-24 17:02:50]|1    |
|1          |[2020-02-24 17:03:10, 2020-02-24 17:03:20]|1    |
|0          |[2020-02-24 17:02:50, 2020-02-24 17:03:00]|2    |
|0          |[2020-02-24 17:03:10, 2020-02-24 17:03:20]|1    |
|0          |[2020-02-24 17:03:00, 2020-02-24 17:03:10]|1    |
+--------